# 모듈 임포트

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import requests
from PIL import Image
import io
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# chromed driver 불러오기

In [2]:
os.chmod('/Users/jeongdaegyun/Desktop/Musinsa/selenium_crawling/chromedriver_mac_arm64/chromedriver', 0o755)

driver = webdriver.Chrome(executable_path="./chromedriver_mac_arm64/chromedriver")

# 무신사 코디맵으로 이동

In [3]:
driver.get("https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1")
driver.implicitly_wait(3)

# 데이터프레임의 한 행에 해당하는 샘플을 추출하는 함수

In [4]:
# num: 한 페이지의 코디는 1~60까지 인덱싱되어있음 
# img_name: 저장할 이미지의 이름
# path: 저장할 경로 (기본은 같은 경로의 image라는 폴더)

def get_sample(num, img_name = None, path = './image'):
    
    # sample이 담길 list
    data = []
    
    # 페이지 이동
    element = driver.find_element(By.XPATH, f'/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{num}]/div[2]/a/strong')
    driver.execute_script("arguments[0].click();", element)
    
    # 제목 추출
    title = driver.find_element(By.XPATH, '//*[@id="style_info"]/h2').text
    
    # 코디에 대한 해시태그 추출
    num_hash = len(driver.find_elements_by_class_name("ui-tag-list__item"))
    hashtag_list = []

    for i in range(1, num_hash+1):
        hashtag_list.append(driver.find_element(By.XPATH, f'//*[@id="style_info"]/div[1]/div/a[{i}]').text[1:])
        
    
    
    if img_name == None:
        raise Exception('img_name 인자가 없습니다!')
    else:    
        #이미지 url 추출
        src = driver.find_element(By.CLASS_NAME, 'photo').get_attribute('src')

        # GET 요청으로 이미지 데이터 가져오기
        response = requests.get(src)

        # 가져온 데이터를 이미지 파일로 변환하기
        image = Image.open(io.BytesIO(response.content))

        # 이미지 파일로 저장하기
        image.save(path+'/'+img_name+'.jpg')
        
        driver.back()
        driver.implicitly_wait(3)
        data.append(title)
        data.append(path+'/'+img_name+'.jpg')
        data.append(hashtag_list)
        
        return data

# 데이터 추출

In [5]:
page_num = 20 # 추출할 페이지 분량 (페이지 당 60개)

In [11]:
for j in range(12, page_num+1):
    driver.get(f"https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page={j}")
    time.sleep(2)
    if j == 12:
        s = 11
    else:
        s = 1
    for i in range(s, 61):
        time.sleep(2)    
        samples.append(get_sample(i, img_name = f'image_{(j-1)*60+i}'))

# ndarray로 바꾸어 형태 확인

In [12]:
samples = np.array(samples)

/var/folders/_r/h97z5hwj0g79ns0tlws44lyw0000gn/T/ipykernel_59157/499109790.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  samples = np.array(samples)


In [13]:
samples.shape

(1200, 3)

# dataframe에 넣기

In [14]:
df = pd.DataFrame(samples, columns = ['title', 'img_path', 'hashtag'])

In [15]:
df.head(6)

,title,img_path,hashtag
0,매일 입을래,./image/image_1.jpg,"[봄, 캐주얼, 긴소매 티셔츠, 데님 팬츠, 마르디 메크르디 슈즈, 맥끌라니, 바우..."
1,센스 있는 조합,./image/image_2.jpg,"[봄, 캐주얼, 긴소매 티셔츠, 데님 팬츠, 마뗑킴, 스타디움 재킷, 아디다스, 에..."
2,매력 만점,./image/image_3.jpg,"[봄, 캐주얼, 데님 팬츠, 메타버스, 반원 아틀리에, 블루종/MA-1, 에코백, ..."
3,우아한 자태,./image/image_4.jpg,"[로맨틱, 봄, 데님 팬츠, 루시크루, 룩캐스트, 목걸이/펜던트, 밸럽, 셔츠/블라..."
4,워너비 스타일,./image/image_5.jpg,"[걸리시, 봄, 귀걸이, 낫노잉, 니트/스웨터, 로씨로씨, 마지셔우드, 미니스커트,..."
5,깔끔한 조합,./image/image_6.jpg,"[골프, 봄, 골프가방, 골프화, 니트/스웨터, 니티나, 데상트 골프, 마르디 메크..."


# 이미지 이름과 인덱스 일치시키기

In [16]:
df.index = [i+1 for i in df.index]

In [17]:
df.head()

,title,img_path,hashtag
1,매일 입을래,./image/image_1.jpg,"[봄, 캐주얼, 긴소매 티셔츠, 데님 팬츠, 마르디 메크르디 슈즈, 맥끌라니, 바우..."
2,센스 있는 조합,./image/image_2.jpg,"[봄, 캐주얼, 긴소매 티셔츠, 데님 팬츠, 마뗑킴, 스타디움 재킷, 아디다스, 에..."
3,매력 만점,./image/image_3.jpg,"[봄, 캐주얼, 데님 팬츠, 메타버스, 반원 아틀리에, 블루종/MA-1, 에코백, ..."
4,우아한 자태,./image/image_4.jpg,"[로맨틱, 봄, 데님 팬츠, 루시크루, 룩캐스트, 목걸이/펜던트, 밸럽, 셔츠/블라..."
5,워너비 스타일,./image/image_5.jpg,"[걸리시, 봄, 귀걸이, 낫노잉, 니트/스웨터, 로씨로씨, 마지셔우드, 미니스커트,..."


# 결측치, 형태 확인

In [18]:
df.isnull().sum()

title       0
img_path    0
hashtag     0
dtype: int64

In [19]:
df.shape

(1200, 3)

In [20]:
df.to_csv('data.csv', sep=',', na_rep='NaN')